In [8]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import timedelta,datetime,date

In [9]:
from config import G_WEB

In [13]:
#state listing
STATES = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
         "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
         "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
         "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
         "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
REGION_ID = "US"
#group search TERM lists
WINERY_L = ["winery","vineyard","wine+spirits","wine+garden"]
DISTILLERY_L = ["distillery","distill+spirit","distiller"]
BREWERY_L = ["brewery","brew+pub","taphouse","beer+garden"]

#combined lists
TERM_SEARCH = winery_l+distillery_l+brewery_l


In [14]:
NAME_DATA = []
LON_DATA = []
LAT_DATA = []
PLACE_ID = []
JSON_URLS = []
STATE_ABR = []
PLACE_SEARCH = []
EST_DATA = []
SEARCHED = []
TIME_SEC = []
# set up a parameters dictionary

# base url
BASE_URL = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
COUNTER = 0
for TERM in TERM_SEARCH:
    START_TIME = time.time()
    print("Query TERMs: "+TERM)
    for STATE in STATES:
        QRY = str(TERM + "+in+" + STATE)
        PARAMS = {"key": G_WEB,"query": QRY,"region":REGION_ID}
        RESPONSE = requests.get(BASE_URL, params = PARAMS)
        PLACE_INFO = RESPONSE.json()
        for res in PLACE_INFO["results"]:
            NAME_DATA.append(res["name"])
            LON_DATA.append(res["geometry"]["location"]["lng"])
            LAT_DATA.append(res["geometry"]["location"]["lat"])
            PLACE_ID.append(res["place_id"])
            STATE_ABR.append(state)
            PLACE_SEARCH.append(TERM)
            EST_DATA.append(TERM)
            JSON_URLS.append(RESPONSE.url)
        #time.sleep(1.5)
    END_TIME = round(time.time()-START_TIME,3)
    COUNTER = COUNTER + 1
    print("        API DATA RETRIEVAL COMPLETE for search term: %s. (elapsed time: %s seconds)" %((TERM), (END_TIME)))
    print("        Total of %s term(s) completed out of %s" %(COUNTER, len(TERM_SEARCH)))
    buffer="-"*COUNTER
    print("        Percentage complete: {0:.1%}".format(round(COUNTER/len(TERM_SEARCH),3)))
    print("        "+buffer)
    SEARCHED.append(TERM)
    TIME_SEC.append(END_TIME)

Query TERMs: winery
        API DATA RETRIEVAL COMPLETE for search TERM: winery. (elapsed time: 40.503 seconds)
        Total of 1 TERM(s) completed out of 11
        Percentage complete: 9.1%
        -
Query TERMs: vineyard
        API DATA RETRIEVAL COMPLETE for search TERM: vineyard. (elapsed time: 42.134 seconds)
        Total of 2 TERM(s) completed out of 11
        Percentage complete: 18.2%
        --
Query TERMs: wine+spirits
        API DATA RETRIEVAL COMPLETE for search TERM: wine+spirits. (elapsed time: 42.865 seconds)
        Total of 3 TERM(s) completed out of 11
        Percentage complete: 27.3%
        ---
Query TERMs: wine+garden
        API DATA RETRIEVAL COMPLETE for search TERM: wine+garden. (elapsed time: 38.239 seconds)
        Total of 4 TERM(s) completed out of 11
        Percentage complete: 36.4%
        ----
Query TERMs: distillery
        API DATA RETRIEVAL COMPLETE for search TERM: distillery. (elapsed time: 35.337 seconds)
        Total of 5 TERM(s) comple

In [15]:
print("Total number of records retrieved: {:,.0f}".format(len(PLACE_ID)))

Total number of records retrieved: 8,709


In [16]:
DATA_OUTPUT = pd.DataFrame(np.column_stack([PLACE_SEARCH,NAME_DATA,STATE_ABR,LON_DATA,LAT_DATA,PLACE_ID,EST_DATA,JSON_URLS]),
                       columns = ["query","name","state","lon","lat","place_id","est","json_url"])


In [17]:
PERF_DF = pd.DataFrame(np.column_stack([SEARCHED,TIME_SEC]),columns = ["query","time_in_sec"])

In [18]:
DATA_OUTPUT = DATA_OUTPUT.drop_duplicates(subset="place_id", keep = "last", inplace = False)
print("Total number of unique records retrieved: {:,.0f}".format(len(DATA_OUTPUT)))

Total number of unique records retrieved: 5,705


In [19]:
START_TIME = time.time()
for i in range(DATA_OUTPUT.est.count()):
    for WL in WINERY_L:
        DATA_OUTPUT.est.i = DATA_OUTPUT.est.replace(
        to_replace = WL,
        value = "winery",
        inplace = True
        )
    for BL in BREWERY_L: 
        DATA_OUTPUT.est.i = DATA_OUTPUT.est.replace(
        to_replace = BL,
        value = "brewery",
        inplace = True
        )
    for DL in DISTILLERY_L:
        DATA_OUTPUT.est.i = DATA_OUTPUT.est.replace(
        to_replace = DL,
        value = "distillery",
        inplace = True
        )
print("elapsed time: %s seconds)" %(round(time.time()-START_TIME,3)))        

elapsed time: 29.787 seconds)


In [20]:
DATA_OUTPUT.set_index("place_id").to_csv("google_sample.csv")

In [21]:
PERF_DF.to_csv("performance_report.csv")